In [1]:
import exifread
import cv2
import os
import numpy as np

In [2]:
def get_exposureTime(imgPath):
    
    with open(imgPath, 'rb') as f:
        tags = exifread.process_file(f, stop_tag='EXIF ExposureTime') 
    timeStr = str(tags['EXIF ExposureTime'].values[0])
    a, b = np.float32(timeStr.split('/'))
    exposureTime = a/b
    
    return exposureTime

In [3]:
def read_image(dirname):
    
    images = []
    exposureTimes = []

    for filename in np.sort(os.listdir(dirname)):
        if os.path.splitext(filename)[1] in ['.JPG']:
            imgPath = os.path.join(dirname, filename)
            im = cv2.imread(imgPath)
            images += [im]
            exposureTimes += [get_exposureTime(imgPath)]

    return images, exposureTimes

In [4]:
images, exposureTimes = read_image('data/set2')
print(exposureTimes)

[0.00013500743, 0.0003030303, 0.00032299742, 0.00065789477, 0.001242236, 0.001996008, 0.0028490028, 0.005, 0.00625, 0.01, 0.0125, 0.022222223, 0.033333335]
